 https://www.kaggle.com/code/rkuo2000/chromadb-langchain-gpt4all

https://python.langchain.com/docs/integrations/llms/gpt4all

https://github.com/nomic-ai/gpt4all/tree/main/gpt4all-bindings/python

https://www.kaggle.com/code/gpreda/test-gpt4all-on-kaggle

In [1]:
import time
start = time.time()

In [2]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2476e51c-6a02-5d59-7830-5546c1d0c4d4)
GPU 1: Tesla T4 (UUID: GPU-b631fb6f-e289-3f17-a672-93854b72c0b5)


In [3]:
%%time

! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers
! pip install -qq -U bitsandbytes bitsandbytes-cuda117
! pip install -qq -U gpt4all

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, b

In [4]:
#!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin models

In [5]:
import warnings
warnings.filterwarnings("ignore")

import os
import textwrap
import shutil
import pandas as pd
import glob

import langchain
from langchain.llms import HuggingFacePipeline

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

#quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

#os.environ['TRANSFORMERS_CACHE'] = 'cache/'

print(langchain.__version__)

0.1.0


In [6]:
### Multi-document retriever
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
#from gpt4all import GPT4All
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [7]:
!wget https://gpt4all.io/models/gguf/wizardlm-13b-v1.2.Q4_0.gguf

--2024-01-06 02:02:37--  https://gpt4all.io/models/gguf/wizardlm-13b-v1.2.Q4_0.gguf
Resolving gpt4all.io (gpt4all.io)... 104.26.1.159, 104.26.0.159, 172.67.71.169, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.1.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7365834624 (6.9G)
Saving to: 'wizardlm-13b-v1.2.Q4_0.gguf'

wizardlm-13b-v1.2.Q 100%[===================>]   6.86G  25.1MB/s    in 6m 6s   

2024-01-06 02:08:43 (19.2 MB/s) - 'wizardlm-13b-v1.2.Q4_0.gguf' saved [7365834624/7365834624]



In [8]:

llm = GPT4All(model="/kaggle/working/wizardlm-13b-v1.2.Q4_0.gguf")

In [9]:
#print(llm.generate("The capital of France is ", max_tokens=1028))

In [10]:
llm

GPT4All(model='/kaggle/working/wizardlm-13b-v1.2.Q4_0.gguf', client=<gpt4all.gpt4all.GPT4All object at 0x79f18021e0b0>)

In [11]:
shutil.copytree('/kaggle/input/mkvc-6-finalmix/kaggle/working/mk-vectordb-chroma-six', '/kaggle/working/mk-vectordb-chroma')

'/kaggle/working/mk-vectordb-chroma'

In [12]:
%%time

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="intfloat/multilingual-e5-base", model_kwargs={"device": "cuda"})

vectordb = Chroma(persist_directory='mk-vectordb-chroma',embedding_function=instructor_embeddings,)
#vectordb.persist()

.gitattributes:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 5.95 s, sys: 5.95 s, total: 11.9 s
Wall time: 35.6 s


In [13]:
%%time
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
#retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25})

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True,
                                       verbose=False)

CPU times: user 976 µs, sys: 0 ns, total: 976 µs
Wall time: 988 µs


In [14]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=GPT4All(model='/kaggle/working/wizardlm-13b-v1.2.Q4_0.gguf', client=<gpt4all.gpt4all.GPT4All object at 0x79f18021e0b0>)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x79f1a285e770>, search_kwargs={'k': 3}))

In [15]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [16]:
def llm_ans(query):
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    return ans

In [17]:
#model

In [18]:
dataset = pd.read_csv('/kaggle/input/mkri-qa-3/dataset.csv',sep=';')
dataset

,no,question,a,b,c,d,e,answer,explained,source,qa_comb
0,1,Berdasarkan Pasal 24C ayat (1) Undang-Undang D...,Mahkamah Agung berwenang mengadili pada tingka...,Mahkamah Konstitusi berwenang mengadili pada t...,Kepolisian Negara Indonesia mengadili pada tin...,Presiden Republik Indonesia mengadili pada tin...,Tentara Negara Indonesia mengadili pada tingka...,b,Mahkamah Konstitusi berwenang mengadili pada t...,3282_putusan_mkri_6516.pdf,Berdasarkan Pasal 24C ayat (1) Undang-Undang D...
1,2,Permohonan diajukan dalam jangka waktu paling ...,"Pasal 70 ayat (3) UU MK, Pasal 111 ayat (2) UU...","Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU...","Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU...","Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU...","Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU...",d,"Bahwa berdasarkan Pasal 74 ayat (3) UU MK, Pas...",3270_putusan_mkri_6508.pdf,Permohonan diajukan dalam jangka waktu paling ...
2,3,Pasal 1 ayat (3) UUD 1945 yang berbunyi…,Negara Indonesia adalah negara Hukum,Negara Indonesia adalah negara Islam,Negara Indonesia adalah negara liberal,Negara Indonesia adalah negara kepulauan,Negara Indonesia adalah negara Kristen,a,"Pasal 1 ayat (3) UUD 1945 yang berbunyi ""Negar...",1338_putusan_sidang_114%20PUU%202012%20-%20KUH...,Pasal 1 ayat (3) UUD 1945 yang berbunyi…\n A. ...
3,4,Bahwa dengan demikian norma Pasal 244 KUHAP te...,Pasal 21D ayat (1) UUD 1945,Pasal 22D ayat (1) UUD 1945,Pasal 23D ayat (1) UUD 1945,Pasal 24D ayat (1) UUD 1945,Pasal 28D ayat (1) UUD 1945,e,Bahwa dengan demikian norma Pasal 244 KUHAP te...,1338_putusan_sidang_114%2520PUU%25202012%2520-...,Bahwa dengan demikian norma Pasal 244 KUHAP te...
4,5,Bunyi rumusan Pasal 244 KUHAP...,Keputusan perkara pidana tidak dapat diberikan...,Terhadap putusan perkara pidana yang diberikan...,Terhadap putusan perkara pidana yang diberikan...,Mengenai kedudukan hukum para Pemohon Pemerint...,NaN,c,"Bunyi rumusan Pasal 244 KUHAP adalah ""Terhadap...",1338_putusan_sidang_114%2520PUU%25202012%2520-...,Bunyi rumusan Pasal 244 KUHAP...\n A. Keputus...
...,...,...,...,...,...,...,...,...,...,...,...
95,96,"Governors, regents and mayors, respectively, a...",1945 Constitution Article 18 paragraph (4),1945 Constitution Article 22E paragraph (1),1945 Constitution Article 27 paragraph (1),1945 Constitution Article 28C paragraph (2),1945 Constitution Article 28D paragraph (1),a,1945 Constitution Article 18 paragraph (4): Go...,526_putusan_sidang_PUTUSAN%2520PUU%2520TOLI-TO...,"Governors, regents and mayors, respectively, a..."
96,97,"General elections are held directly, publicly,...",1946 Constitution Article 18 paragraph (4),1946 Constitution Article 22E paragraph (1),1946 Constitution Article 27 paragraph (1),1946 Constitution Article 28C paragraph (2),1946 Constitution Article 28D paragraph (1),b,1945 Constitution Article 22E paragraph (1): G...,526_putusan_sidang_PUTUSAN%2520PUU%2520TOLI-TO...,"General elections are held directly, publicly,..."
97,98,All citizens have the same position under the ...,1947 Constitution Article 18 paragraph (4),1947 Constitution Article 22E paragraph (1),1947 Constitution Article 27 paragraph (1),1947 Constitution Article 28C paragraph (2),1947 Constitution Article 28D paragraph (1),c,1945 Constitution Article 27 paragraph (1): Al...,526_putusan_sidang_PUTUSAN%2520PUU%2520TOLI-TO...,All citizens have the same position under the ...
98,99,Everyone has the right to advance themselves i...,1948 Constitution Article 18 paragraph (4),1948 Constitution Article 22E paragraph (1),1948 Constitution Article 27 paragraph (1),1948 Constitution Article 28C paragraph (2),1948 Constitution Article 28D paragraph (1),d,1945 Constitution Article 28C paragraph (2): E...,526_putusan_sidang_PUTUSAN%2520PUU%2520TOLI-TO...,Everyone has the right to advance themselves i...


In [19]:
%%time
for x,y in zip(dataset.no,dataset.qa_comb):
    llm_time_start = time.time()
    print("Nomor: ",x)
    print(y)
    llm_ans(y)
    llm_time_end = time.time()
    print("Time: ",llm_time_end - llm_time_start)
    print("==============================================")

Nomor:  1
Berdasarkan Pasal 24C ayat (1) Undang-Undang Dasar Negara Republik Indonesia Tahun 1945 (selanjutnya disebut UUD 1945) menyatakan bahwa...\n A.  Mahkamah Agung berwenang mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\n B.  Mahkamah Konstitusi berwenang mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\n C.  Kepolisian Negara Indonesia mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\n D.  Presiden Republik Indonesia mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum\n E.  Tentara Negara Indonesia mengadili pada tingkat pertama dan terakhir yang putusannya bersifat final, antara lain, untuk memutus pe

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Pasal 24C ayat (1) UUD 1945 menyatakan bahwa Mahkamah Agung berwenang mengadili pada tingkat pertama dan
terakhir yang putusannya bersifat final, antara lain, untuk memutus perselisihan tentang hasil pemilihan umum.


Sources:
Time:  329.19601035118103
Nomor:  2
Permohonan diajukan dalam jangka waktu paling lama 3 X 24 (tiga kali dua puluh empat) jam sejak diumumkan penetapan perolehan suara hasil Pemilu Anggota DPR dan DPRD secara nasional oleh KPU, Isi dari pernyataan ini ada di pasal…\n A.  Pasal 70 ayat (3) UU MK, Pasal 111 ayat (2) UU Pemilu, dan Pasal 6 ayat (1) PMK Nomor 2 Tahun 2018\n B.  Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU Pemilu, dan Pasal 2 ayat (1) PMK Nomor 2 Tahun 2011\n C.  Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU Pemilu, dan Pasal 1 ayat (1) PMK Nomor 2 Tahun 2018\n D.  Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU Pemilu, dan Pasal 6 ayat (1) PMK Nomor 2 Tahun 2018\n E.  Pasal 74 ayat (3) UU MK, Pasal 474 ayat (2) UU Pemilu, dan Pasal 4 ayat (1) PMK 

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The pieces of context provided are related to the Indonesian legal system and specifically refer to
provisions in various laws that govern elections. They include references to the General Elections Law (UU
MK), the Regional Elections Law (UU Pemilu), and Presidential Decree No. 2 Year 2018 (PMK Nomor 2 Tahun 2018).

The question asks for the section of a specific law that pertains to a certain timeframe after the
announcement of national election results by KPU, or General Elections Commission. The answer choices provide
different sections from various laws as potential answers.


Sources:
Time:  334.46122002601624
Nomor:  3
Pasal 1 ayat (3) UUD 1945 yang berbunyi…\n A.  Negara Indonesia adalah negara Hukum\n B.  Negara Indonesia adalah negara Islam\n C.  Negara Indonesia adalah negara liberal\n D.  Negara Indonesia adalah negara kepulauan\n E.  Negara Indonesia adalah negara Kristen


...

 The answer is B. 1947 Constitution Article 22E paragraph (1).


Sources:
Time:  107.9377269744873
Nomor:  99
Everyone has the right to advance themselves in fighting for their rights collectively to develop their society, nation and state, as stated in article...\n A.  1948 Constitution Article 18 paragraph (4)\n B.  1948 Constitution Article 22E paragraph (1)\n C.  1948 Constitution Article 27 paragraph (1)\n D.  1948 Constitution Article 28C paragraph (2)\n E.  1948 Constitution Article 28D paragraph (1)


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The answer is B.


Sources:
Time:  98.3470778465271
Nomor:  100
Every person has the right to recognition, guarantees, protection and fair legal certainty as well as equal treatment before the law, as stated in Article...\n A.  1949 Constitution Article 18 paragraph (4)\n B.  1949 Constitution Article 22E paragraph (1)\n C.  1949 Constitution Article 27 paragraph (1)\n D.  1949 Constitution Article 28C paragraph (2)\n E.  1949 Constitution Article 28D paragraph (1)


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The answer is B.


Sources:
Time:  99.68164801597595
CPU times: user 19h 15min 47s, sys: 56.1 s, total: 19h 16min 43s
Wall time: 4h 56min 9s


In [20]:
end = time.time()
print(end - start)

18460.947495937347
